# Homework 1
#### Author: Prof. Gustavo Soares

#### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import bql
bq = bql.Service()

#### Introduction

You are a investor considering investing in the following ETFs:

* **iShares US Property Yield UCITS ETF (IDUP LN Equity)** which is an exchange-traded fund incorporated in Ireland. The Fund aims to track the performance of FTSE EPRA/NAREIT US Dividend+ Index. The fund is denominated and trades in USD but on European time.
* **Xtrackers FTSE Developed Europe Real Estate UCITS ETF (D5BK B2 Equity)** which physically replicates the performance of the FTSE EPRA/NAREIT Developed Europe Real Estate Net Return Index. The ETF offers direct investment in European Real Estate. Provides exposure to Real Estate holding and development companies, as well as REITs. The fund is denominated and trades in EUR during European time.
* **iShares Japan REIT ETF (1476 JT Equity)** which is an exchange-traded fund established in Japan.  The  fund's objective is to provide investment results that correspond to the  performance of the TSE REIT index. The fund is denominated and trades in JPY during European time.

Let's grab their total returns from Bloomberg:

In [2]:
request = "get(TOTAL_RETURN_INDEX(DATES=RANGE('-10Y', '0D'))) for(['IDUP LN Equity','1476 JT Equity','D5BK B2 Equity'])"
res = bq.execute(request)
etf_df = bql.combined_df(res)
etf_df.to_csv('etf.csv')
etf_df.head()

,DATE,"TOTAL_RETURN_INDEX(DATES=RANGE('-10Y','0D'))"
ID,,
IDUP LN Equity,2011-04-29,1.0
IDUP LN Equity,2011-04-30,1.0
IDUP LN Equity,2011-05-01,1.0
IDUP LN Equity,2011-05-02,1.0
IDUP LN Equity,2011-05-03,1.0


## Q1: Excess returns in local currency

Assume you are a local currency based investor, i.e., you care about the returns of these three ETFs in the currency in which they trade, calculate the daily excess returns of each of the assets and show some descriptive stats for these three investments. For this question, you will need the risk free rate in each market, let's use the 3M libor rates from Bloomberg:

In [4]:
RATES_DICT = {
    'US' : 'US0003M Index',
    'JP' : 'JY0003M Index',
    'EU' : 'EUR003M Equity'
}

tickers = list(RATES_DICT.values())
fields = {'price' : bq.data.px_last(fill='PREV', start=etf_df['DATE'].min(), end=etf_df['DATE'].max())}
req = bql.Request(tickers, fields)
res = bq.execute(req)
rates_df = bql.combined_df(res)
rates_df.to_csv('rates.csv')
rates_df.head()

,DATE,CURRENCY,price
ID,,,
US0003M Index,2011-04-29,,0.27300
US0003M Index,2011-04-30,,0.27300
US0003M Index,2011-05-01,,0.27300
US0003M Index,2011-05-02,,0.27300
US0003M Index,2011-05-03,,0.27225


## Q2: Excess returns in BRL

Assume now you are a BRL based investor, i.e., you care about the returns of these three ETFs in BRL, calculate the daily excess returns of each of the assets and show some descriptive stats for these three investments. For this question, you will need the BRL risk free rate and the relevant exchange rates (make sure you use the exchange rate correctly!). Let's grab that from Bloomberg:

In [5]:
FX_DICT = {
    'CDI' : 'BZACCETP Index',
    'US' : 'BRL Curncy',
    'JP' : 'JPY Curncy',
    'EU' : 'EUR Curncy'
}

tickers = list(FX_DICT.values())
fields = {'price' : bq.data.px_last(fill='PREV', start=etf_df['DATE'].min(), end=etf_df['DATE'].max())}
req = bql.Request(tickers, fields)
res = bq.execute(req)
fx_df = bql.combined_df(res)
fx_df.to_csv('fx.csv')
fx_df.head()

,DATE,CURRENCY,price
ID,,,
BZACCETP Index,2011-04-29,BRL,25.39454
BZACCETP Index,2011-04-30,BRL,25.39454
BZACCETP Index,2011-05-01,BRL,25.39454
BZACCETP Index,2011-05-02,BRL,25.40589
BZACCETP Index,2011-05-03,BRL,25.41722


## Q3: Efficient frontier

Assume now you are a BRL based investor and construct a chart with the [efficient frontier](https://en.wikipedia.org/wiki/Efficient_frontier) for those investments using the long-term expected returns and standard deviation as estimates. Choose a portfolio in the efficient frontier that you think is good and highlight it in the chart. When you estimate the assets covariance matrix, think about their different trading time. Do you think that plays a role on making correlations higher or smaller? Is there a way to mitigate the impact of different trading hours on correlations?

## Q4: Capital market line

Assume now you are a BRL based investor but you have access to unfunded (swaps) instruments. That is, you can trade on those ETFs without posting any cash. Add the [capital market line](https://en.wikipedia.org/wiki/Capital_market_line) to the chart above. Choose a portfolio in the capital market line that you think is good and highlight it in the chart.